# Практическое задание №6 по теме "Взаимосвязь величин. Показатели корреляции. Корреляционный анализ. Проверка на нормальность".

Дополнительное задание

In [1]:
import numpy as np
from scipy import stats

## Задача 1

Даны значения величины заработной платы заемщиков банка (`salary`) и значения их поведенческого кредитного скоринга (`scoring`):
```
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
```
Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек `numpy`, `pandas` и др.) найдите коэффициент корреляции Пирсона двух выборок.

__Решение__

Найдем коэффициент корреляции Пирсона, который можно посчитать следующим образом:

$$r_{XY} = \dfrac{\sigma_{XY}}{\sigma_X \cdot \sigma_Y},$$

где $\sigma_X$, $\sigma_Y$ — среднее квадратическое отклонение, $\sigma_{XY}$ - ковариация (несмещенная оценка).

$$\sigma_{XY} = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n \left( x_i - \overline{X} \right) \cdot \left( y_i - \overline{Y} \right)$$

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
n = len(scoring) if len(scoring)  == len(salary)  else 0

In [3]:
salary_mean = sum(salary)/n
scoring_mean = sum(scoring)/n

salary_mean, scoring_mean

(101.4, 709.9)

In [4]:
cov = sum([(scoring[i] - scoring_mean)*(salary[i] - salary_mean) for i in range(n)])/(n-1)
cov

10175.377777777778

In [5]:
salary_std = (sum([(s - salary_mean)**2 for s in salary])/(n-1))**0.5
scoring_std = (sum([(s - scoring_mean)**2 for s in scoring])/(n-1))**0.5

salary_std, scoring_std

(62.31318747531162, 183.99544076477068)

In [6]:
corr = cov/salary_std/scoring_std
round(corr, 8)

0.88749009

__Ответ__: $\sigma_{XY}$ = 0.88749009

---

## Задача 2

Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:
* Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
* Статистика: `t = r * sqrt(n - 2) / sqrt(1 - r ** 2)`, где `r` - коэффициент корреляции Пирсона, посчитанный по выборке.
* В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром `df = n - 2`.

В качестве ответа запишите 1, если нулевая гипотеза верна, и 0 иначе.

__Решение__

1. В качестве нулевой гипотезы $H_{0}$ берется гипотеза о том, что реальный коэффициент корреляции Пирсона равен `0`

2. По условию задания выбрана следующая статистика: `t = r * sqrt(n - 2) / sqrt(1 - r ** 2)`, где `r` - коэффициент корреляции Пирсона, посчитанный по выборке.

3. Возьмем уровень значимости $\alpha = 0.05$

4. Критическая область $\Omega_\alpha$

Выбранная по условию задания статистика имеет имеет __распределение Стьюдента__ или __t-распределение__ с параметром $df = n - 2$. Также нам известно, что альтернативная гипотеза является двухсторонней, а это значит, что критическая область также будет двухсторонней. Для её построения нам понадобятся квантили $t_{\alpha / 2}$ и $t_{1 - \alpha / 2}$ t-распределения. Найдём их с помощью `scipy.stats.t.ppf`.

In [7]:
def t_statistic(n: int, r: float) -> float:
    return (r * np.sqrt(n - 2) / np.sqrt(1 - r ** 2))

In [8]:
alpha = 0.05

In [9]:
t1 = stats.t.ppf(alpha/2, df=n-2)
t2 = stats.t.ppf(1-alpha/2, df=n-2)

np.round((t1, t2), 8)

array([-2.30600414,  2.30600414])

Итак, критическая область:
$$\Omega_\alpha = (-\infty, -2.30600414) \cup (2.30600414, \infty)$$

5. Статистический тест

In [10]:
S = t_statistic(n, corr)
S

5.447168150485575

Найденное значение попало в критическую область $\Omega_\alpha = (-\infty, -2.30600414) \cup (2.30600414, \infty)$. Нулевау гипотезу стоит отвергнуть. 

Для чистоты найдем __P-значение__:

In [11]:
p_left = stats.t.cdf(S, df=n - 2)
p_right = 1 - stats.t.cdf(S, df=n - 2)

pvalue = 2 * min(p_left, p_right)
print(f"P-value: {pvalue}")

P-value: 0.0006107546587257318


__P-значение__ приблизительно равно нулю, это означает, что сколько бы мы не уменьшали $\alpha$ (уровень значимости), не попасть в критическую область довольно сложность. Нулевую гипотезу $H_{0}$ смело можно отвергать.

__Ответ__: 0

---

## Задача 3

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:
```
131, 125, 115, 122, 131, 115, 107, 99, 125, 111
```
Известно, что в генеральной совокупности IQ распределен нормально. Найдите ширину доверительного интервала для математического ожидания с надежностью 0.95.

__Ответ__: 15.0878327

__Решение__

Генеральная совокупность распределена нормально, дисперсия неизвестна, оценка математического ожидания. На основании этих данных используем __t-статистику__, которая имеет распределение Стьюдента:
$$t = \dfrac{\overline{X} - \mu}{\sigma_X / \sqrt{n}}$$

Требуется построить доверительный интервал для математического ожидания $\mu$ с доверительной вероятностью $p=0.95$

$$P \left( \overline{X} + t_{\alpha / 2, \: n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{1 - \alpha / 2, \: n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \right) = p$$

In [12]:
IQ = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])
p = 0.95

Расчитаем $\overline X$, $\sigma_{X}$, $n$, $alpha$

In [13]:
n = IQ.shape[0]
mean = IQ.mean()
std = IQ.std(ddof=1)
alpha = 1 - p

n, mean, std, alpha

(10, 118.1, 10.54566788359614, 0.050000000000000044)

Найдем квантили:

In [14]:
t1 = stats.t.ppf(alpha/2, df=n-1)
t2 = stats.t.ppf(1-alpha/2, df=n-1)

t1, t2

(-2.2621571627409915, 2.2621571627409915)

Итак, доверительный интервал:

In [15]:
CI = (mean + t1 * std / np.sqrt(n), mean + t2 * std / np.sqrt(n))
CI

(110.55608365158724, 125.64391634841274)

Ширина доверительного интервала:

In [16]:
np.round(CI[1] - CI[0], 8)

15.0878327

__Ответ__: 15.0878327

---